In [1]:
%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt

from pylabnet.network.core.generic_server import GenericServer 
from pylabnet.utils.logging.logger import LogService, LogClient

In [2]:
try:
    log_server.stop()
except NameError:
    pass

log_service = LogService(
    log_output=True,
    name="output",
    dir_path="C:/Users/Qi/Desktop/"
)

log_server = GenericServer(
    service=log_service, 
    host='localhost',
    port=5520
)

log_server.start()

In [3]:
log_client = LogClient(
    host='localhost', 
    port=5520, 
    module_tag='random')

2020-09-23 18:44:22,783 - INFO - Client connected
2020-09-23 18:44:22,790 - INFO -  random: Started logging


In [4]:
from pylabnet.network.core.service_base import ServiceBase
from pylabnet.network.core.client_base import ClientBase

class AbstractDriver:
    """
    Handles the "device" functions. This is directly interfaced by the device server.
    """

    def __init__(self, logger, init_value):
        self.logger = logger
        self.value = init_value

    def function(self):
        self.value *= 2
        self.logger.info(f"The value is now {self.value}")

class AbstractService(ServiceBase):
    """
    Interfaces with the device by directly calling the device driver functions. Listens for commands from clients that use the exposed wrapper functions.
    """
    def exposed_function(self):
        """
        Wrapper function that clients call. This then goes on to call the driver function that connects to the device. """
        return self._module.function()

class AbstractClient(ClientBase):
    """
    Connects to the device server. Gives commands by calling the wrapper functions that are exposed by the server, so it does not have direct access to the driver functions.
    """
    def function(self):
        """
        Function for client users to call. This calls a wrapper function exposed by the server which is itself also a wrapper for the driver.
        """
        return self._service.exposed_function()


In [5]:
# Driver sets up the device state
abstract_driver = AbstractDriver(logger=log_client, init_value=2)

# Service will hold the device driver and will be hosted by the device server
abstract_service = AbstractService()
abstract_service.assign_module(module=abstract_driver)
abstract_service.assign_logger(logger=log_client)

In [6]:
try:
    abstract_server.stop()
except NameError:
    pass

abstract_server = GenericServer(
    service=abstract_service, 
    host='localhost',
    port=1234
)
abstract_server.start()

In [7]:
abstract_client = AbstractClient(
    host='localhost',
    port=1234
)
abstract_client.function()

2020-09-23 18:46:18,510 - INFO -  random: Client connected
2020-09-23 18:46:18,514 - INFO -  random: The value is now 4


In [8]:
abstract_client.function()

2020-09-23 18:46:25,616 - INFO -  random: The value is now 8


In [9]:
abstract_client.function()
abstract_client.function()
abstract_client.function()
abstract_client.function()

2020-09-23 18:46:31,997 - INFO -  random: The value is now 16
2020-09-23 18:46:32,001 - INFO -  random: The value is now 32
2020-09-23 18:46:32,005 - INFO -  random: The value is now 64
2020-09-23 18:46:32,009 - INFO -  random: The value is now 128


In [12]:
abstract_server.stop()
log_server.stop()